In [9]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyspark.sql.functions as F
import setuptools
import statsmodels.api as sm
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from sklearn import linear_model, preprocessing
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
%matplotlib inline

load_dotenv()

True

In [10]:
df = pd.read_parquet('data/preprocessed_regression_data')
df.head()

,index,registration_year,n_doors,n_seats,body_type,municipality,vtax,price,electric
0,34360122116,2011,4,5,AF,785,165.71,30942.385802,0
1,34360136446,2011,4,5,AC,211,218.27,48674.575258,0
2,34360139666,2011,4,5,AF,636,208.05,44547.810689,0
3,34360139668,2011,4,5,AF,434,208.05,44547.810689,0
4,34360152903,2011,4,5,AF,609,182.13,26603.686706,0


In [11]:
print(df.shape[0])
df = df.dropna()
print(df.shape[0])

1039372
1036833


In [18]:
y = df['electric']
X = df[['vtax', 'price']]

pipe = Pipeline([
    ('scaler', StandardScaler()), # zero mean unit variance
    #('polyd2', PolynomialFeatures(degree=2))
])

Xp = pipe.fit_transform(X)
#Xp = pd.DataFrame(Xp, columns=pipe.named_steps['polyd2'].get_feature_names_out())
#Xp.head()

fit = pipe.fit(Xp, y)

smfit = sm.Logit(y, sm.add_constant(Xp)).fit()
smfit.summary()

Optimization terminated successfully.
         Current function value: 0.239181
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               electric   No. Observations:              1036833
Model:                          Logit   Df Residuals:                  1036830
Method:                           MLE   Df Model:                            2
Date:                Mon, 28 Oct 2024   Pseudo R-squ.:                  0.1631
Time:                        16:54:23   Log-Likelihood:            -2.4799e+05
converged:                       True   LL-Null:                   -2.9632e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8479      0.005   -600.025      0.000      -2.857      -2.839
x1             0.9520      0.004    230.914      0.000       0.944       0.960
x2             0.4789      0.003    146.948      0.000       0.472       0.485
==============================================================================
"""

In [20]:
smfit.pvalues

const    0.0
x1       0.0
x2       0.0
dtype: float64